# So, you want to learn deep learning the hard way?


## Getting started

First, you'll need to install torch, torchvision, ipympl, ipywidgets

In [7]:
%matplotlib widget
from matplotlib.pyplot import *
from matplotlib import animation as ani
import ipywidgets as widgets
import numpy as np

## Deep learning in a nutshell

Optimization is where you have a function and you find some parameters that find an optimum (e.g. a minimum) of the function. Machine learning is where you get to choose the function and it contains a lot of data. Deep learning is where you pick a function that isn't special in any mathematical way, use gradient descent and hope for the best.

It turns out that this works quite well.



## Optimization and gradient descent

A popular function for illustrating optimization algorithms is the [Rosenbrock Banana Function](https://en.wikipedia.org/wiki/Rosenbrock_function). The equation is:
$$
    f = (a-x)^2 + b(y-x^2)^2
$$
which in code is:

In [2]:
def rosenbrock(x, y=1, a=1, b=100):
    return (a-x)**2 + (y - x**2)**2 * b


In [3]:
xs = np.arange(-2, 2, 0.1)
ys = [rosenbrock(x) for x in xs]
plot(xs, ys)
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The simplest way to minimize this function is *gradient descent*. This is a very simple algorithm, where you simply walk down hill by going in the opposite direction of the gradient. you need a scale $\alpha$ because the magnitute of the gradient may be large or small, so the steps may not be of a convenient size.  In maths this is:
$$
    x_{n+1} = x_n - \alpha\left.\frac{\text{d}f(x)}{\text{d}x}\right|_{x=x_n}.
$$
In deep learning, $\alpha$ is known as the *learning rate*.

Conceptually that is simple, but annoying because you need the derivative. In the case of Rosenbrock, the equation is not too bad:
$$
    \frac{\text{d}f}{\text{d}x} = -2(a-x) - 4bx(y-x^2)
$$

In [5]:
def rosenbrock_deriv_x(x, y=1, a=1, b=100):
    return -2*(a-x) - 4*b*x*(y-x**2)

In [8]:
alpha = 0.0001
#Start at x=-2
x_n = -2
fig = figure()
axes = fig.add_subplot(111)

def draw_graph(_=None):
    global x_n, alpha
    axes.clear()
    axes.plot(xs, ys)
    axes.plot(x_n, rosenbrock(x_n), 'r*')
    
    #Compute the next position using gradient descent
    next_x = x_n - alpha * rosenbrock_deriv_x(x_n)
    
    #Plot some stuff
    y = rosenbrock(x_n)
    y_update = y - (x_n-next_x) * rosenbrock_deriv_x(x_n)
    axes.plot([x_n, next_x],  [y, y_update], 'g')
    axes.plot([next_x, next_x], [y_update, rosenbrock(next_x)], 'k:')
    legend(['Function', 'Current x', 'Gradient'])
    title(f"y = {y}")
    
    #Update the current x position to the new one
    x_n = next_x
    show()

draw_graph()
button = widgets.Button(description="Click me to update")
output = widgets.Output()
display(button, output)
button.on_click(draw_graph)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Click me to update', style=ButtonStyle())

Output()

## Automatic differentiation

Computing gradients by hand is tedious and very error prone. Fortunately there is a technique to do it automatically called *automatic differentiation*. This technique is not an automatic way of computing derivatives symbolically, it's a numeric technique based on repeated application of the chain rule (and a few others such as the product rule). This techinque forms the core of how pytorch works.


Let's consider some simple operations we might want to do. A simple one is:
$$
y = g(x)^2
$$
and it's derivative is:
$$
    \frac{\text{d}y}{\text{d}x} = 2*g(x) \frac{\text{d}g}{\text{d}x}
$$
The important observation is that if we already have $g(x)$ and $\frac{\text{d}g}{\text{d}x}$, then we can easily compute $\frac{\text{d}y}{\text{d}x}$. This is how autodiff works. Instead of just using numbers, we can use tuples that hold a number and it's derivative. So we can implement a function to square one of these,which is just an implementation of the above maths.

In [7]:
def square(n):
    return (n[0]*n[0], 2*n[0]*n[1])

We can do the same for other mathematical operations too. For example:
$$
    y = f(x) + g(x)
$$
and
$$
      \frac{\text{d}y}{\text{d}x} = \frac{\text{d}f}{\text{d}x} + \frac{\text{d}g}{\text{d}x}
$$
which in code looks like:

In [8]:
def add(left, right):
    return (left[0] + right[0], left[1] + right[1])

So far so good, but how do we get started? Well, we start using a variable and it's derivative. The derivative of something with respect to itself is just 1, which is nice and simple. So if we wanted to start with $x=5$, we would just write:

In [9]:
x = (5, 1)

We can now use the functions we wrote. We only have squaring and adding so far, but that's enough for a function like this:
$$
    y = (x^2 + x)^2 + x
$$
so if we use our new functions, it computes not just $y$ but its derivative automatically:

In [76]:
y = add(square(add(square(x), x)), x)
print(y)

(905, 661)


Let's verify this. The symboic derivative is:
$$
    y =  2 (x^2 + x) (2x+1) + 1
$$
and running the numbers gives:

In [77]:
x=5
print((x**2+x)**2 + x)
print(2*(x**2 + x)*(2*x+1)+1)

905
661


Woo hoo!

Using tuples and functions is hard to read because it doesn't use infix notation. We can fix this by using a class instead of a tuple and defining operators for the class. Here is an implementation of just enough of the operators to execute the Rosenbrock function. Feel free to extend it:

In [10]:
import math
class AutoDiff:
    def __init__(self, value, gradient = 1):
        self.value = value
        self.gradient = gradient
        
    def __add__(self, rhs):
        if isinstance(rhs, AutoDiff):
            return AutoDiff(self.value + rhs.value, self.gradient + rhs.gradient)
        else:
            return self + AutoDiff(rhs,0)
        
    def __rsub__(self, lhs):
        return self + lhs*-1
    
    def __mul__(self, rhs):
        if isinstance(rhs, AutoDiff):
            return AutoDiff(self.value * rhs.value, self.gradient * rhs.value + self.value * rhs.gradient)
        else:
            return self * AutoDiff(rhs, 0)
        
    def __pow__(self, exponent):
        a = self.value
        
        if isinstance(exponent, AutoDiff):
            b = exponent.value
            return AutoDiff(a**b, b*a**(b-1)*self.gradient + a**b * math.log(a) * exponent.gradient)
        else:
            b = exponent
            return self ** AutoDiff(exponent,0)
        
    def __str__(self):
        return str((self.value, self.gradient))

We can then pass this to `rosenbrock` instead of a number and it will compute the value and it's derivative:

In [79]:
x = AutoDiff(2)
print(rosenbrock(x))

(901, 2402.0)


And double checking to be sure:

In [80]:
print(rosenbrock(2), rosenbrock_deriv_x(2))

901 2402


It works again! Mostly you won't need to know about this. However sometimes details will leak into your program, often in the forms of bugs. If you acciently convert your variable from an `AutoDiff` type to a number and use it, you can lose derivatives for some of your code without noticing, for example. Sometimes you need to do this intentionally.

## The same again but using pytorch

Torch essentially does all of this but it is much more powerful and sophisticated and complete, so it often requires a couple more steps. In addition Torch works entirely with what it calls tensors (multidimensional arrays), so everything must be in terms of those even if you are using scalars.

In [10]:
import torch as t

#Declare a variable and enable automatic differentation
x = t.tensor([2.0], requires_grad=True)
y = rosenbrock(x)
#Perform computation of the derivative
y.backward()

print(y.item())
#Because we called y.backward(), this is the derivative of y with respect to x
print(x.grad.item())


901.0
2402.0


PyTorch uses a slightly different algorithm to do automatic differentiation. You need to flag which variables you want gradients for (by using `requires_grad=True`) and then the result is computed when `backwards()` is called and the results are stored in the variable. This makes it much easier to figure out which derivative belongs to which dependent variable when there's more than one.

Since pytorch can compute gradients, we can use it for gradient descent. Here is the same gradient descent algorithm implemented in pytorch:

In [11]:
figure()
plot(xs, ys)

x_n = t.tensor([-2.0])

for i in range(20):
    #We want to compute gradients with respect to x_n 
    x_n.requires_grad=True
    y_n = rosenbrock(x_n)
    # the item() attribute converts a single-element tensor into a float (i.e. a number that can be used)
    plot(x_n.item(), y_n.item(), 'r*')

    # This is our "automatic differentiation", which calculates the gradient
    y_n.backward()
    
    # Now we don't want to compute gradients with respect to x_n, because
    # in the equation after that would mean x_n has gradients with respect
    # to previous versions of itself, which is not how gradient descent works.
    #
    # In other words, the new version of x_n is a simple scalar with no derivatives.
    # Derivatives are then computed in the next iteration.
    x_n.requires_grad=False
    x_n = x_n - alpha * x_n.grad

print(x_n.item())
    
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-1.0686041116714478


## A more complex example

In [3]:
import torch
#import torchvision
#Image size
size = 40

#These are the x coordinates for every pixel
numbers = list(range(0, size))
square = [numbers for _ in numbers]
x_coordinates = torch.tensor(square, dtype = torch.float)

The following computes values of a Gaussian for the input. It assumes that `xs` corresponds to a square grid of pixels and contains only their $x$ coordinates, i.e. 
```
0 1 2 3 4 
0 1 2 3 4
0 1 2 3 4
0 1 2 3 4
0 1 2 3 4
```
`pos` is the position as a Torch tensor and `sigma` is self explanatory.

Torch is an array maths library, which means it supports operations such as scalar times array (both of course are called Tensors in Torch terminology, a scalar being a tensor containing one element), 

In [20]:
def gaussian(xs, pos, sigma):
    #Generate Y coordinates
    ys = xs.permute([1,0])
    return torch.exp(-((pos[0] - xs)**2 + (pos[1]-ys)**2)/(2*sigma**2)) / (2*math.pi*sigma**2)

We can use this to generate some simulated data, for example an image containing a Gaussian blob and some noise:

In [25]:
import math
data = gaussian(x_coordinates, torch.tensor([25.5, 10.0]), 3.5) + torch.normal(torch.zeros([size,size]), 0.0005)
figure()
imshow(data)
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can find the position of this spot by fitting a Gaussian using least squares. This is an optimization procedure where we find the parameters of the Gaussian that minimize the sum of square errors. The sum of square errors being the thing we're minimizing is known as the *loss function*. Since it's an optimization problem we can solve it using gradient descent in Pytorch.

In [26]:
#Now define a Gaussian spot to be optimised
position = torch.tensor([10., 10.])
sigma = torch.tensor([10.0])

#Learning rates are always problem dependent. This rate of 1000 works 
#for this problem but is unusually large. Ususally learning rates are much less than 1.
learning_rate=1000

figure()
def update_graph(_):
    global position, sigma, learning_rate
    #We want to optimize position and size
    position.requires_grad = True
    sigma.requires_grad = True
    
    #The model is also a single Gaussian
    model = gaussian(x_coordinates, position, sigma)
    #The sum square errors loss
    loss = ((data-model)**2).sum()
    
    #Gradient descent
    loss.backward()
    position.requires_grad = False
    sigma.requires_grad = False
    
    position = position - position.grad * learning_rate
    sigma = sigma - sigma.grad * learning_rate

    clf()
    subplot(1,2, 1)
    imshow(data.numpy())
    title("data")
    subplot(1,2, 2)
    imshow(model.detach().numpy())
    title("model")
    suptitle(f"position = {list(position.detach().numpy())}, sigma={sigma.item()}")
    
update_graph(None)
button = widgets.Button(description="Click me a lot")
output = widgets.Output()
display(button, output)
button.on_click(update_graph)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Click me a lot', style=ButtonStyle())

Output()

## Pytorch optimizers

Writing gradient descent is quite repetitive, and it's the simplest of the gradient descent algorithms. Pytorch has number of optimizers built. Two of note are SGD and Adam. SGD is the classic stochastic gradient descent algorithm, which is gradient descent with momentum. Adam is another good choice, and no one will ever reject your paper for picking either of these two.

Using optimizers is straightforward:

In [27]:
#Now define a Gaussian spot to be optimised
position = torch.tensor([ 10., 10.], requires_grad=True)
sigma = torch.tensor([10.0], requires_grad=True)
learning_rate=1000

figure()
optimizer = torch.optim.SGD([position, sigma], lr = 1000)

def update_graph(_):
    global position, sigma

    #pytorch will do very odd things if you forget this
    optimizer.zero_grad()
    
    model = gaussian(x_coordinates, position, sigma)
    loss = ((data-model)**2).sum()
    
    loss.backward()
    optimizer.step()
    
    clf()
    subplot(1,2, 1)
    imshow(data.numpy())
    title("data")
    subplot(1,2, 2)
    imshow(model.detach().numpy())
    title("model")
    suptitle(f"position = {list(position.detach().numpy())}, sigma={sigma.item()}")
    
update_graph(None)
button = widgets.Button(description="Click me a lot")
output = widgets.Output()
display(button, output)
button.on_click(update_graph)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Click me a lot', style=ButtonStyle())

Output()